In [1]:
!nvidia-smi

Sun Nov 12 03:13:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/archive.zip

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: fire&smoke/yolo/0000018730.txt  
  inflating: fire&smoke/yolo/0000018731.txt  
  inflating: fire&smoke/yolo/0000018732.txt  
  inflating: fire&smoke/yolo/0000018733.txt  
  inflating: fire&smoke/yolo/0000018734.txt  
  inflating: fire&smoke/yolo/0000018735.txt  
  inflating: fire&smoke/yolo/0000018736.txt  
  inflating: fire&smoke/yolo/0000018737.txt  
  inflating: fire&smoke/yolo/0000018738.txt  
  inflating: fire&smoke/yolo/0000018739.txt  
  inflating: fire&smoke/yolo/0000018740.txt  
  inflating: fire&smoke/yolo/0000018741.txt  
  inflating: fire&smoke/yolo/0000018742.txt  
  inflating: fire&smoke/yolo/0000018743.txt  
  inflating: fire&smoke/yolo/0000018744.txt  
  inflating: fire&smoke/yolo/0000018745.txt  
  inflating: fire&smoke/yolo/0000018746.txt  
  inflating: fire&smoke/yolo/0000018747.txt  
  inflating: fire&smoke/yolo/0000018748.txt  
  inflating: fire&smoke/yolo/0000018749.txt  
  inflating: fire&smoke

In [4]:
# Pip install method (recommended)

!pip install ultralytics==8.0.20

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
Setup complete ✅ (8 CPUs, 51.0 GB RAM, 31.4/166.8 GB disk)


In [5]:
import os

# Đường dẫn đến thư mục chứa dữ liệu
dataset_path = "/content/fire&smoke"

# Đổi tên thư mục "yolo" thành "labels"
os.rename(os.path.join(dataset_path, "yolo"), os.path.join(dataset_path, "labels"))


In [6]:
import os
import random
from shutil import copyfile

# Set your dataset path
dataset_path = "/content/fire&smoke"

# Set the paths for the train, val, and test folders
train_image_path = os.path.join(dataset_path, "train", "images")
train_yolo_path = os.path.join(dataset_path, "train", "labels")

val_image_path = os.path.join(dataset_path, "val", "images")
val_yolo_path = os.path.join(dataset_path, "val", "labels")

test_image_path = os.path.join(dataset_path, "test", "images")
test_yolo_path = os.path.join(dataset_path, "test", "labels")

# Create the train, val, and test folders if they don't exist
for folder in [train_image_path, train_yolo_path, val_image_path, val_yolo_path, test_image_path, test_yolo_path]:
    if not os.path.exists(folder):
        os.makedirs(folder)

# Set the ratio for train, val, test
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Get a list of all image files in the dataset
image_files = [f for f in os.listdir(os.path.join(dataset_path, "images")) if f.endswith((".jpg", ".png"))]

# Shuffle the list of image files
random.shuffle(image_files)

# Calculate the number of images for each set
num_images = len(image_files)
num_train = int(train_ratio * num_images)
num_val = int(val_ratio * num_images)

# Split the dataset
train_files = image_files[:num_train]
val_files = image_files[num_train:num_train + num_val]
test_files = image_files[num_train + num_val:]

# Copy images to the train folder
for file in train_files:
    image_path = os.path.join(dataset_path, "images", file)
    copyfile(image_path, os.path.join(train_image_path, file))

# Copy images to the val folder
for file in val_files:
    image_path = os.path.join(dataset_path, "images", file)
    copyfile(image_path, os.path.join(val_image_path, file))

# Copy images to the test folder
for file in test_files:
    image_path = os.path.join(dataset_path, "images", file)
    copyfile(image_path, os.path.join(test_image_path, file))

# Copy YOLO annotation files to the train folder
for file in train_files:
    yolo_path = os.path.join(dataset_path, "labels", file.replace(os.path.splitext(file)[-1], ".txt"))
    copyfile(yolo_path, os.path.join(train_yolo_path, file.replace(os.path.splitext(file)[-1], ".txt")))

# Copy YOLO annotation files to the val folder
for file in val_files:
    yolo_path = os.path.join(dataset_path, "labels", file.replace(os.path.splitext(file)[-1], ".txt"))
    copyfile(yolo_path, os.path.join(val_yolo_path, file.replace(os.path.splitext(file)[-1], ".txt")))

# Copy YOLO annotation files to the test folder
for file in test_files:
    yolo_path = os.path.join(dataset_path, "labels", file.replace(os.path.splitext(file)[-1], ".txt"))
    copyfile(yolo_path, os.path.join(test_yolo_path, file.replace(os.path.splitext(file)[-1], ".txt")))


In [8]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolov8s.pt')  # Use your custom YAML file for model configuration

# Train the model using your custom YAML configuration
model.train(data='data.yaml', imgsz=640, batch=8, device=0)

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla V100-SXM2-16GB, 16151MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=data.yaml, epochs=100, patience=50, batch=8, imgsz=640, save=True, cache=False, device=0, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opse